# Обучение модели

### Установка и импорт неообходимых модулей

In [2]:
!pip install pandas sklearn catboost tqdm

In [3]:
import numpy as np
import pickle
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Загрузка данных

In [4]:
import pandas as pd
import re
df = pd.read_csv("uncoded_data.csv")
target = 'price'

## Препроцессинг

###  Кодирование нечисловых значений

In [5]:
les = {}
for name in tqdm(df.columns):
    if df[name].dtypes == object or df[name].dtypes == bool:
        df[name].fillna("-9999999")
        les[name] = preprocessing.LabelEncoder()
        df[name] = les[name].fit_transform(df[name].apply(str))

100%|██████████| 17/17 [00:00<00:00, 1725.93it/s]


In [6]:
df = df.dropna()

#### Сохранение моделей кодировшиков для разкодирования в будуюшем

In [7]:
with open('les.pkl', 'wb') as output:
    pickle.dump(les, output)

# Обучение модели

Разделим датасет на тестовую и тренировчную выборку для независимого тестирования точности модели

In [8]:
from sklearn.model_selection import train_test_split
target_data = df[target].values
features_data = df.drop(target,axis=1).values
x_train, x_test, y_train, y_test = train_test_split(features_data, target_data, test_size=0.15)

In [9]:
# Подключение библиотек с моделями
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier

Подберем параметры которые максиально улучшат и оптимизируют процесс обучения,  
создадим экземпляры моделей с этими параметрами

Обучим три модели и сравним их показатели

In [10]:
# Создание экземпляров классификаторов
RFC = RandomForestClassifier()
CBC = CatBoostClassifier(iterations=10)
KNC = KNeighborsClassifier()

In [12]:
KNC.fit(x_train, y_train)
CBC.fit(x_train, y_train)
RFC.fit(x_train, y_train)

ValueError: Unknown label type: 'continuous'

Точность из вывода выше, это проценты в нормализованном виде, т.е. умножив получившееся числа на 100% мы получим точность в процентах.

In [11]:
# Вывод результатов
#print('Точность градиентного бустинга:{}'.format(CBC.score(x_test, y_test)))
print('Точность метода случайных деревьев:{}'.format(RFC.score(x_test, y_test)))
#print('Точность метода ближайших соседей:{}'.format(KNC.score(x_test, y_test)))

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Посмотрев на результаты тестирования моделей можно сказать, что метод градиентного бустинга и метода сучайных деревьев очень схожи по точности, а точность метода ближайших соседей заметно меньше остальных двух.  Схожесть методов градиентного бустинга и случайных деревьев можно обьяснить, концептуальной схожестью алгоритмов: оба строят логические деревья.

<h1 align="center">Feature engenering</h1>

Я предположу, что достав из краткого описания не одно, а несколько самых популярных слов, я увеличу точночть алгоритма

In [37]:
reeng_feach = pd.DataFrame()

stat = {}
for i in df['Автор']:
    if i not in stat:
        stat[i] = 1
    else:
        stat[i] += 1
stat = list(stat)
autor = []
for i in df['Автор']:
    autor.append(stat.index(i))
reeng_feach['Автор'] = autor

stat = {}
for i in df['Адресат']:
    if i not in stat:
        stat[i] = 1
    else:
        stat[i] += 1
stat = list(stat)
adres = []
for i in df['Адресат']:
    adres.append(stat.index(i))
reeng_feach['Адресат'] = adres


stat = {}
for i in df['Организация']:
    if i not in stat:
        stat[i] = 1
    else:
        stat[i] += 1
stat = list(stat)
org = []
for i in df['Организация']:
    org.append(stat.index(i))
reeng_feach['Организация'] = org



dic = {}
for i in df['Краткоесодержание']:
    for j in i:
        if j not in dic:
            dic[j] = 1
        else:
            dic[j] += 1


def get_pop(m):
    t = 0
    temp = []
    for i in m:
        if dic[i] > t:
            temp.append(i)
            t = dic[i]
            if len(temp) >= 4:
                temp = temp[1:]
    while len(temp) < 3:
        temp.append('')
    return temp

sod = []
for i in df['Краткоесодержание']:
    sod.append(get_pop(i))
key1 = []
key2 = []
key3 = []
for i in sod:
    key1.append(i[0])
    key2.append(i[1])
    key3.append(i[2])
reeng_feach['Keyword1'] = key1
reeng_feach['Keyword2'] = key2
reeng_feach['Keyword3'] = key3

stat = {}
for i in reeng_feach['Keyword1']:
    if i not in stat:
        stat[i] = 1
    else:
        stat[i] += 1
stat = list(stat)
soderj = []
for i in reeng_feach['Keyword1']:
    soderj.append(stat.index(i))
reeng_feach['Keyword1'] = soderj

stat = {}
for i in reeng_feach['Keyword2']:
    if i not in stat:
        stat[i] = 1
    else:
        stat[i] += 1
stat = list(stat)
soderj = []
for i in reeng_feach['Keyword2']:
    soderj.append(stat.index(i))
reeng_feach['Keyword2'] = soderj

stat = {}
for i in reeng_feach['Keyword3']:
    if i not in stat:
        stat[i] = 1
    else:
        stat[i] += 1
stat = list(stat)
soderj = []
for i in reeng_feach['Keyword3']:
    soderj.append(stat.index(i))
reeng_feach['Keyword3'] = soderj
reeng_feach.head()

,Автор,Адресат,Организация,Keyword1,Keyword2,Keyword3
0,0,0,0,0,0,0
1,1,0,1,1,1,1
2,2,0,2,2,2,1
3,3,1,3,1,1,1
4,4,0,4,1,1,1


После того реинженеринга фич, надо снова разделить датасет на тренировочую и тестовую часть.  
После чего заново обучить модели и посмотреть на результат

In [38]:
from sklearn.model_selection import train_test_split
target_data = df[target].values
features_data = reeng_feach.drop(target,axis=1).values
x_train, x_test, y_train, y_test = train_test_split(features_data, target_data, test_size=0.15)

In [39]:
# Создание экземпляров классификаторов
RFC = RandomForestClassifier()
CBC = CatBoostClassifier(logging_level="Silent")
KNC = KNeighborsClassifier()

In [40]:
# Обучение моделей
KNC.fit(x_train, y_train)
CBC.fit(x_train, y_train)
RFC.fit(x_train, y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [41]:
# Вывод результатов
print('Точность градиентного бустинга:{}'.format(CBC.score(x_test, y_test)))
print('Точность метода случайных деревьев:{}'.format(RFC.score(x_test, y_test)))
print('Точность метода ближайших соседей:{}'.format(KNC.score(x_test, y_test)))

Точность градиентного бустинга:0.5040214477211796
Точность метода случайных деревьев:0.4839142091152815
Точность метода ближайших соседей:0.4061662198391421


Точность из вывода выше, это проценты в нормализованном виде, т.е. умножив получившееся числа на 100% мы получим точность в процентах.

Как видно из вывода выше, реинженеринг фич помог увеличить точность моей целевой модели (градиентный бустинг) на 3%. На таком небольшом наборе данных это очень солидный прирост точности, даже не смотря на то, что точность метода ближайших соседей уменьшилась. Все вышеперечисленное доказывает, что выбраная мной модель действительно самая подходящая для данных задач

In [42]:
import pickle

In [45]:
with open('model.pic','wb') as f:
    pickle.dump(CBC,f)
    f.close()

In [82]:
stat = {}
for i in df['Адресат']:
    if i not in stat:
        stat[i] = 1
    else:
        stat[i] += 1

In [83]:
stat = list(stat)
stat
with open('names.pic','wb') as f:
    pickle.dump(stat,f)
    f.close()

In [84]:
stat

['Уразов Р.Н.',
 'Иванюк Л.А.',
 'Миронова С.В.',
 'Тымчиков А.Ю.',
 'Крайчинская С.Б.',
 'Антонов Ю.В.',
 'Глушко Д.Е.',
 'Аштаева К.Д.',
 'Митькина А.Н.',
 'Желковский И.И.',
 'Митяева И.В.',
 'Мухаметзянов Д.Ф.',
 'Недоспасова О.В.Никул Е.В.',
 'Саликова К.В.',
 'Куколев С.В.',
 'Носков К.Ю.Попова А.Ю.Кочнев Д.В.Алешин А.В.Колобков П.А.Колокольцев В.А.Золотов В.В.Булавин В.И.Бортников А.В.Чайка Ю.Я.Скворцова В.И.Зиничев Е.Н.Шойгу С.К.Васильева О.Ю.Дитрих Е.И.Кобылкин Д.Н.Уразов Р.Н.Нарышкин С.Е.Нерадько А.В.Чиханчин Ю.А.Кудрин А.Л.Лавров С.ВБастрыкин А.И.Коновалов А.В.',
 'Гонтарь М.А.',
 'Недоспасова О.В.',
 'Глушко Д.Е.Тымчиков А.Ю.',
 'Глушко Д.Е.Иванюк Л.А.',
 'Митькина А.Н.Миронова С.В.',
 'Серафимова О.В.',
 'Селукова А.И.',
 'Зайчикова И.А.',
 'Зензинова К.В.',
 'Кожевникова Е.И.',
 'Базер О.Э.',
 'Митяева И.В.Крайчинская С.Б.',
 'Досканова А.В.',
 'Вишнякова О.Б.',
 'Никул Е.В.',
 'Алимов А.А.',
 'Уфимцев Д.А.',
 'Судаков К.Ю.',
 'Митяева И.В.Судаков К.Ю.',
 'Лошкарева Е.П.'